<a href="https://colab.research.google.com/github/Juhwan01/DeepDive/blob/main/Bert_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98(%EC%A7%81%EC%A0%91_%EA%B5%AC%ED%98%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [ ]:
# 네이버 영화 리뷰 데이터 학습을 위해 훈련 데이터와 테스트 데이터를 다운로드합니다.
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [ ]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

In [ ]:
train_data.drop_duplicates(subset=['document'], inplace = True)
train_data = train_data.dropna(how = 'any')
print('훈련 데이터의 리뷰 수:',len(train_data))

In [ ]:
test_data = test_data.dropna(how = 'any')
print('테스트 데이터의 리뷰 수:',len(test_data))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

In [ ]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

In [ ]:
# 인코딩하면서 특수 토큰이 들어가고 다시 디코딩 하니 특수 토큰이 기존의 문장에서 추가되서 나옴(문장 복원 과정)
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

In [ ]:
# 특수 토큰 이름과 ID 출력
print(tokenizer.cls_token,':', tokenizer.cls_token_id)
print(tokenizer.sep_token,':', tokenizer.sep_token_id)
print(tokenizer.pad_token,':', tokenizer.pad_token_id)

In [ ]:
max_length = 128

# 버전 차이로 인한 함수 변화
# encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화",max_seq_length=max_seq_length, pad_to_max_length=True)
# max_sequence_length -> max_length // pad_to_max_length=True -> padding = 'max_length'
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화",max_length = max_length, padding='max_length')
print(encoded_result)
print('길이 :', len(encoded_result))

In [ ]:
print([0]*max_length)

In [ ]:
# 어텐션 마스크 인코딩
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (max_length - valid_num) * [0])

In [ ]:
def convert_examples_to_features(examples, labels, max_length, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    # tqdm은 여기서 진행률 표시줄을 추가하기 위해서 사용한다
    # (examples, labels)로 묶고, examples의 길이만큼 반복한다 그걸 zip 객체를 감싸 진행바 표시
    # !!! truncation=True 옵션 추가 128 넘으면 자동으로 잘라 -> 원본 코드 돌려보니 128을 넘는 문장이 있기 때문
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_length, padding='max_length',truncation=True)

        # 단어 위치하면 1, 패딩은 0으로
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_length - padding_count) + [0] * padding_count

        # 세그먼트 인코딩 = 입력 [CLS] 나는 학생이다 [SEP] 너는 선생님이다 [SEP] ->
        #            Segment ID:   0     0     0       0    1       1        1
        token_type_id = [0] * max_length

        # assert -> 조건 만족하지 않으면 에러 발생시키는 디버킹 도구
        # input_id의 길이가 최대 시퀀스 길이와 같은지 확인
        # 만약 다르면 에러 메시지와 함께 실제 길이와 기대 길이를 출력
        assert len(input_id) == max_length, "Error with input length {} vs {}".format(len(input_id), max_length)

        # attention_mask의 길이가 최대 시퀀스 길이와 같은지 확인
        # attention_mask는 어떤 토큰이 실제 데이터이고 어떤 토큰이 패딩인지 표시
        assert len(attention_mask) == max_length, "Error with attention mask length {} vs {}".format(len(attention_mask), max_length)

        # token_type_id의 길이가 최대 시퀀스 길이와 같은지 확인
        # token_type_id는 BERT에서 첫 번째/두 번째 문장을 구분하는 데 사용 (0 또는 1)
        assert len(token_type_id) == max_length, "Error with token type length {} vs {}".format(len(token_type_id), max_length)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    # 리스트를 그대로 쓰면 파이썬 내장 자료구조라 연산 느리고 벡터연산도 불가능
    # -> 리스트+리스트 연결만되기 때문
    # 따라서 배열끼리 더하고 곱하는 것도 자연스러운 numpy 배열로 바꿔서 숫자를 효율적으로 처리하려고 변환을 한다
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    # 라벨을 정수 배열로 변환해서 분류 문제에 적합하게 만듬
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels


In [ ]:
train_X, train_y = convert_examples_to_features(train_data['document'],train_data['label'],max_length=max_length,tokenizer=tokenizer)

In [ ]:
test_X, test_y = convert_examples_to_features(test_data['document'],test_data['label'],max_length=max_length,tokenizer=tokenizer)

In [ ]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 :', attention_mask)
print('세그먼트 인코딩 :', token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :', tokenizer.decode(input_id))
print('레이블 :', label)


In [ ]:
# PyTorch 형식으로 저장된 'klue/bert-base' 모델을
# TensorFlow 형식으로 변환해서 불러오기 위해 from_pt=True 옵션 사용
model = TFBertModel.from_pretrained("klue/bert-base",from_pt=True)

In [ ]:
max_length = 128

# 입력 토큰 ID들을 받는 Input 레이어 (배열 길이는 max_length, 데이터 타입은 32비트 정수)
input_ids_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)

# 어텐션 마스크를 받는 Input 레이어 (패딩 등에서 어느 토큰에 집중할지 표시, max_length 길이, int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)

# 토큰 타입 ID를 받는 Input 레이어 (문장 A, B 구분용, max_length 길이, int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)

# BERT 모델에 정석 방식으로 입력-> 원본 입력은 첫번째 입력이 [input_ids_layer]로 감싸져 있어야함
outputs = model(
    input_ids=input_ids_layer,
    attention_mask=attention_masks_layer,
    token_type_ids=token_type_ids_layer
)

In [ ]:
# 모든 토큰의 출력 shape: (batch_size, sequence_length, hidden_size) → 예: (32, 128, 768)
# 보면 128개의 출력이 나오는게 된다 -> 각출력 768차원 벡터 -> many to many
print(outputs[0])

In [ ]:
# 최종 768차원짜리 출력 하나 나옴 -> many to one이 된다.
print(outputs[1])

In [ ]:
# tf.keras.model을 상속받아 사용 -> 상속받은 클래스의 compile,fit 같은 메소드를 사용할 수 있다.
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        # 파이썬3부터는 이렇게 간추려 쓸 수 있음 super(TFBertForSequenceClassification, self).__init__() -> super().__init__()
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        # 출력층을 하나 만들어서 classifier에 저장
        # 출력 뉴런 하나로 이진 분류 예측
        self.classifier = tf.keras.layers.Dense(1,
                                                # 가중치 초기화 방식
                                                # 정규분포에서 값을 뽑되 극단값은 버리는 버전
                                                # BERT 논문에 따르면, 모든 Dense 레이어는 TruncatedNormal(std=0.02)로 초기화해야 학습이 안정적이기 때문
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                # 이진분류 활성화 함수
                                                activation='sigmoid',
                                                name='classifier')
    def call(self, inputs):
        # 입력 받은 3개의 튜플 요소
        input_ids, attention_mask, token_type_ids = inputs
        # 모델에 입력값 전달
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 이진분류 값을 받을거니 [1]번 선택
        # 문장의 대표 벡터인 [CLS] 토큰의 벡터 가져오기
        cls_token = outputs[1]
        # [CLS] 벡터를 Dense 레이어에 넣어서 최종 결과 예측
        # sigmoid이기 때문에 0~1사이의 확률
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
model = TFBertForSequenceClassification("klue/bert-base")

In [ ]:
optimizer = tf.keras.optimizers.Adam(5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=2, batch_size=64, validation_split=0.2)

In [ ]:
results = model.evaluate(test_X,test_y, batch_size=1024)
print("test loss, test acc: ", results)

In [ ]:
def sentiment_predict(new_sentence):
  input_id = tokenizer.encode(new_sentence, max_length=max_length, padding='max_length',truncation=True)
  padding_count = input_id.count(tokenizer.pad_token_id)
  attention_mask = [1]*(max_length-padding_count) + [0]*padding_count
  token_type_id = [0]*max_length

  # 파이썬 리스트 구조 -> 처리 빠른 넘파이 배열 구조로
  input_ids = np.array([input_id])
  attention_masks = np.array([attention_mask])
  token_type_ids = np.array([token_type_id])

  encoded_input = [input_ids,attention_masks,token_type_ids]
  # model.predict는 배치 단위로 예측 결과를 반환함
  # many-to-one 문제라면 출력 형태는 (batch_size, output_dim)
  # many-to-many 문제라면 (batch_size, seq_len, output_dim) 형태임

  # 따라서 output[0]은 배치 첫 번째 샘플의 결과이고
  # output[0][0]은 (many-to-one일 때) 첫 번째 출력값 또는
  # (many-to-many일 때) 첫 번째 토큰의 출력값임

  # 이진 분류에서는 output_dim=1 이므로 output[0][0]으로 확률 값을 얻음
  # output은 (배치 크기, 시퀀스 길이, 출력 차원)의 3차원 배열임
  # output[0]은 배치의 첫 번째 샘플에 해당하는 시퀀스 전체 출력이고,
  # output[0][0]은 그 샘플 시퀀스 중 첫 번째 토큰에 대한 출력 벡터임
  # 마찬가지로 output[1]은 두 번째 샘플의 전체 시퀀스 출력임
  score = model.predict(encoded_input)[0][0]

  if score > 0.5 :
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')